In [ ]:
using JuMP
using GLPK

In [ ]:
function ejDiet()
    categories = ["calories", "protein", "fat", "sodium"]
    category_data = Containers.DenseAxisArray([
        1800 2200;
        91   +Inf;
        0    65;
        0    1779
        ], categories, ["min", "max"]
    )

    foods = [
        "hamburger", "chicken", "hot dog", "fries", "macaroni", "pizza",
        "salad", "milk", "ice cream"
    ]
    cost = Containers.DenseAxisArray(
        [2.49, 2.89, 1.50, 1.89, 2.09, 1.99, 2.49, 0.89, 1.59],
        foods
    )
    
    food_data = Containers.DenseAxisArray([
            410 24 26 730;
            420 32 10 1190;
            560 20 32 1800;
            380  4 19 270;
            320 12 10 930;
            320 15 12 820;
            320 31 12 1230;
            100  8 2.5 125;
            330  8 10 180
        ], foods, categories
    )
    
    model = JuMP.Model(GLPK.Optimizer)
    
    @variable(model, buy[foods] >= 0)
    @variable(model, category_data[c, "min"] <= nutrition[c in categories] <= category_data[c, "max"])

    @objective(model, Min, cost' * buy)
    
    @constraint(model, r[c in categories],
        sum(food_data[f, c] * buy[f] for f in foods) == nutrition[c])

    println(model)
    
    optimize!(model)

    @show JuMP.termination_status(model)
    @show JuMP.primal_status(model)
    @show JuMP.objective_value(model)
    buyval = JuMP.value.(buy)
    for v in eachindex(buyval)
        println(foods[v], " = ", buyval[v])
    end

    return
end

In [ ]:
ejDiet()